# Install Packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Flatten, Activation, GlobalAvgPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
import itertools
import shutil
import os
import random
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import datetime
from time import time

# Using GPU

In [2]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Variable

In [3]:
TRAIN_PATH = './data/seg_train/seg_train/'
VALID_PATH = './data/seg_test/seg_test/'
TEST_PATH = './data/seg_pred/'
IMAGE_SIZE = (224, 224, 3)
BATCH_SIZES = 64
EPOCHS = 80
switch = "mobilenet" #test model

# Data Preparetion

In [4]:
datagen = ImageDataGenerator()
train_batches = datagen.flow_from_directory(directory=TRAIN_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=True,batch_size=BATCH_SIZES)
valid_batches = datagen.flow_from_directory(directory=VALID_PATH, color_mode="rgb",target_size=IMAGE_SIZE[:-1], class_mode="categorical", shuffle=False,batch_size=BATCH_SIZES)
test_batches = datagen.flow_from_directory(directory=TEST_PATH,target_size=IMAGE_SIZE[:-1],color_mode="rgb",batch_size=BATCH_SIZES,class_mode=None,shuffle=False)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Found 7301 images belonging to 1 classes.


# Model Construct

In [5]:
if switch == "my_mobilenet":
    model = myMobileNetV1(input_shape = IMAGE_SIZE, num_classes = 6 ) 
else:
    if switch == "mobilenet":
        md = keras.applications.MobileNet(input_shape = IMAGE_SIZE, weights = "imagenet", include_top = False)
    elif switch == "mobilenetv2":
        md = keras.applications.MobileNetV2(input_shape = IMAGE_SIZE, weights=None)
    elif switch == "ResNet50":
        md = keras.applications.ResNet50(input_shape = IMAGE_SIZE, weights=None)
    x = GlobalAvgPool2D(name='global_avg')(md.output)
    prediction = Dense(6, activation='softmax')(x)
    model = Model(inputs=md.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128       
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 112, 112, 32)      0     

In [6]:
optimizer = Adam(learning_rate = 1e-3)
model.compile(
    loss='categorical_crossentropy',
    optimizer= optimizer,
    metrics=['accuracy']
)
stop_callback = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience = 4, verbose=1, min_lr = 1e-6)
t1 = time()
history = model.fit(train_batches, 
                    validation_data = valid_batches, 
                    epochs= EPOCHS, 
                    steps_per_epoch=len(train_batches), 
                    validation_steps=len(valid_batches), 
                    callbacks = [stop_callback],
                    shuffle = True)
t2 = time()
print("execution time: ", t2 - t1)
model.save_weights('./model/mobile_imagenetweights.h5')

Epoch 1/80
220/220 [==============================] - 54s 228ms/step - loss: 0.5193 - accuracy: 0.8414 - val_loss: 0.3196 - val_accuracy: 0.9027
Epoch 2/80
220/220 [==============================] - 49s 220ms/step - loss: 0.1779 - accuracy: 0.9385 - val_loss: 0.6618 - val_accuracy: 0.8050
Epoch 3/80
220/220 [==============================] - 49s 223ms/step - loss: 0.1374 - accuracy: 0.9527 - val_loss: 0.3095 - val_accuracy: 0.9023
Epoch 4/80
220/220 [==============================] - 48s 218ms/step - loss: 0.1152 - accuracy: 0.9598 - val_loss: 0.4395 - val_accuracy: 0.8480
Epoch 5/80
220/220 [==============================] - 49s 222ms/step - loss: 0.0930 - accuracy: 0.9681 - val_loss: 0.4523 - val_accuracy: 0.8730
Epoch 6/80
220/220 [==============================] - 49s 221ms/step - loss: 0.0872 - accuracy: 0.9701 - val_loss: 0.3885 - val_accuracy: 0.8863
Epoch 7/80
220/220 [==============================] - 49s 222ms/step - loss: 0.0748 - accuracy: 0.9758 - val_loss: 0.4670 - val_ac

# Result

In [7]:
scoreSeg = model.evaluate(valid_batches)
print("Test Data Accuracy = ",scoreSeg[1])

47/47 [==============================] - 3s 63ms/step - loss: 0.4379 - accuracy: 0.9150
Test Data Accuracy =  0.9150000214576721


In [8]:
from sklearn.metrics import classification_report
test_labels = valid_batches.classes 
predictions = model.predict(valid_batches, verbose=1)
y_pred = np.argmax(predictions, axis=-1)
print(classification_report(test_labels, y_pred, target_names = valid_batches.class_indices))

47/47 [==============================] - 3s 63ms/step
              precision    recall  f1-score   support

   buildings       0.95      0.88      0.91       437
      forest       0.99      0.97      0.98       474
     glacier       0.87      0.87      0.87       553
    mountain       0.90      0.84      0.87       525
         sea       0.92      0.95      0.93       510
      street       0.88      0.98      0.93       501

    accuracy                           0.92      3000
   macro avg       0.92      0.92      0.92      3000
weighted avg       0.92      0.92      0.91      3000

